### Prerequisites

In [1]:
%load_ext autoreload
%autoreload 2

import os
import shutil
import glob
import numpy as np
import networkx as nx
from copy import deepcopy
import json

import torch
from GraphTranslatorModule import GraphTranslatorModule
from encoders import TimeEncodingOptions
from graph_visualizations import visualize_conditional_datapoint
from reader import RoutinesDataset


/home/maithili/.conda/envs/pyml/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class Graph():
    def __init__(self, root='kitchen'):
        self.root = root
        self.parents = {}
        self.category = {root:'Rooms'}
    
    def add(self, node, parent, category='placable_object'):
        assert parent in self.parents.keys() or parent == self.root
        self.parents[node] = parent
        self.category[node] = category

    def draw(self):
        G = nx.DiGraph()
        G.add_nodes_from(list(self.parents.keys()))
        G.add_edges_from([(n, self.parents[n]) for n in self.parents])
        still_remaining = deepcopy(list(self.parents.keys()))
        nodelists = [[self.root]]
        while(len(still_remaining)):
            new_layer = []
            remaining_nodes = still_remaining
            still_remaining = []
            for n in remaining_nodes:
                if self.parents[n] in nodelists[-1]:
                    # print(n)
                    new_layer.append(n)
                else:
                    still_remaining.append(n)
            nodelists.append(new_layer)
            # print(new_layer)
        nx.draw_networkx(G)
    
    def move(self, object, location):
        assert location in self.parents.keys() or location == self.root
        self.parents[object] = location
    
    def to_dict(self):
        id_map = {self.root:0}
        id_map .update({n:i+1 for i,n in enumerate(self.parents.keys())})
        graph_dict = {}
        graph_dict['nodes'] = [{'id':i, 'class_name':n, 'category':self.category[n]} for n,i in id_map.items()]
        graph_dict['edges'] = [{'from_id':id_map[n], 'to_id':id_map[p], 'relation_type':'INSIDE'} for n,p in self.parents.items()]
        return graph_dict

    def dump(self, filename):
        graph_dict = self.to_dict()
        with open (filename, 'w') as f:
            json.dump(graph_dict, f, indent=4)

    def num_nodes(self):
        return len(self.parents) + 1
    
    def get_edges(self, node_name, node_id):
        edges = np.zeros((len(node_name), len(node_name)))
        for i,node in enumerate(node_name):
            if node != self.root:
                edges[i,node_id[self.parents[node]]] = 1
        return torch.Tensor(edges).unsqueeze(0)


In [ ]:
def print_changes(edges1, edges_list, node_name):
    new_locs = {}
    for edges2 in edges_list:
        loc1 = edges1.argmax(-1).reshape(-1)
        loc2 = edges2.argmax(-1).reshape(-1)
        for i,(l1, l2) in enumerate(zip(loc1, loc2)):
            if l1 != 0 and l1 != l2:
                if node_name[i] not in new_locs:
                    new_locs[node_name[i]] = node_name[l2]
                    print(f'Object {node_name[i]} moved from {node_name[l1]} to {node_name[l2]}')

In [ ]:
import random
def d(dmin, dmax):
    return (random.random()*(dmax-dmin))+dmin

### Generating data

In [ ]:
initial_graph = Graph('kitchen')
initial_graph.add(node='shelf', parent='kitchen', category='Furniture')
initial_graph.add(node='drawer', parent='kitchen', category='Furniture')
initial_graph.add(node='sink', parent='kitchen', category='Furniture')
initial_graph.add(node='counter', parent='kitchen', category='Furniture')
initial_graph.add(node='fridge', parent='kitchen', category='Furniture')
initial_graph.add(node='table', parent='kitchen', category='Furniture')

initial_graph.add(node='cereal', parent='shelf')
initial_graph.add(node='bowl', parent='drawer')
initial_graph.add(node='pills', parent='drawer')
initial_graph.add(node='banana', parent='counter')
initial_graph.add(node='glass', parent='counter')
initial_graph.add(node='milk', parent='fridge')
# print(initial_graph.parents)
initial_graph.draw()

In [ ]:
initial_graph.dump('demo_graph.json')

In [ ]:
def execute_routine(graph, initial_time, steps):
    time = initial_time
    graph_list = [deepcopy(graph.to_dict())]
    time_list = [initial_time]
    for ((obj, loc), (dmin, dmax)) in steps:
        duration = d(dmin, dmax)
        graph.move(obj, loc)
        time += duration
        graph_list.append(deepcopy(graph.to_dict()))
        time_list.append(deepcopy(time))
    obj_in_use = [[] for _ in range(len(graph_list))]
    return {"graphs":graph_list, "times": time_list, "objects_in_use":obj_in_use}


breakfast_routine_steps = [
        (('cereal', 'table'), (60,90)),
        (('milk', 'table'), (3,5)),
        (('bowl', 'table'), (3,5)),
        (('glass', 'table'), (3,5)),
        (('milk', 'fridge'), (10,15)),
        (('bowl', 'sink'), (10,15)),
        (('glass', 'sink'), (3,5)),
        (('cereal', 'shelf'), (3,5))
    ]
    
medication_steps = [
        (('pills', 'table'), (60,90)),
        (('glass', 'table'), (3,5)),
        (('glass', 'sink'), (10,15)),
        (('pills', 'drawer'), (3,5))
]

In [ ]:
major_breakfast_prob = 1
directory = 'data/demo_routines'
if os.path.exists(directory):
    shutil.rmtree(directory)

os.makedirs(directory)

inf = {"dt": 5, 
"num_train_routines": 50, 
"num_test_routines": 10, 
"weekend_days": [], 
"start_time": 0, 
"end_time": 500, 
"interleaving": False, 
"only_used_objects": True, 
"num_nodes": initial_graph.num_nodes(), 
"search_object_ids": [], 
"search_object_names": []}

with open(os.path.join(directory,"info.json"), 'w') as f:
    json.dump(inf,f)
with open(os.path.join(directory,"classes.json"), 'w') as f:
    classes_dict = {'nodes':initial_graph.to_dict()['nodes'], 'edges':['INSIDE','ON']}
    json.dump(classes_dict,f)

os.makedirs(os.path.join(directory,'routines_train'))
for i in range(inf["num_train_routines"]):
    print(initial_graph.parents)
    routine = execute_routine(initial_graph, inf["start_time"], breakfast_routine_steps+medication_steps)
    
    with open(os.path.join(directory,'routines_train',"{:3d}.json".format(i)), 'w') as f:
        json.dump(routine, f)

os.makedirs(os.path.join(directory,'routines_test'))
for i in range(inf["num_test_routines"]):
    routine = execute_routine(initial_graph, inf["start_time"], breakfast_routine_steps+medication_steps)
    
    with open(os.path.join(directory,'routines_test',"{:3d}.json".format(i)), 'w') as f:
        json.dump(routine, f)



In [ ]:
# ! [ -d data/demo_routines_simple/processed ] || ~/.conda/envs/pyml/bin/python3 ./readerFileBased.py --path=data/demo_routines_simple
# ! ~/.conda/envs/pyml/bin/python3 ./run.py --path=data/demo_routines_simple --name=ours --tags=demo --train_days=50 --logs_dir=logs_demo_simple/50 --write_ckpt

### Evaluate model

In [ ]:
with open('data/demo_routines/processed/common_data.json') as f:
    common_data = json.load(f)

node_name = common_data['node_classes']
node_id = {n:i for i,n in enumerate(node_name)}

standard_nodes = torch.Tensor(np.eye(len(node_name))).unsqueeze(0)

In [ ]:
log_dir = 'logs/demo_routines/demo_routines/ours_5epochs'

with open (os.path.join(log_dir,'config.json')) as f:
    cfg = json.load(f)

time_encoding_func = TimeEncodingOptions()(cfg['TIME_ENCODING'])

data=cfg['DATA_PARAM']
checkpoint_file = glob.glob(log_dir+'/*.ckpt')[0]


model = GraphTranslatorModule.load_from_checkpoint(checkpoint_file, 
                                                    num_nodes=data['n_nodes'],
                                                    node_feature_len=data['n_len'],
                                                    context_len=data['c_len'],
                                                    edge_importance=cfg['EDGE_IMPORTANCE'],
                                                    edge_dropout_prob = cfg['EDGE_DROPOUT_PROB'],
                                                    tn_loss_weight=cfg['TN_LOSS_WEIGHT'],
                                                    learned_time_periods=cfg['LEARNED_TIME_PERIODS'],
                                                    hidden_layer_size=cfg['HIDDEN_LAYER_SIZE'])

model.load_state_dict(torch.load(os.path.join(log_dir,'weights.pt')))

In [ ]:
graph = Graph('kitchen')

graph.add(node='shelf', parent='kitchen', category='Furniture')
graph.add(node='drawer', parent='kitchen', category='Furniture')
graph.add(node='sink', parent='kitchen', category='Furniture')
graph.add(node='counter', parent='kitchen', category='Furniture')
graph.add(node='fridge', parent='kitchen', category='Furniture')
graph.add(node='table', parent='kitchen', category='Furniture')

graph.add(node='cereal', parent='shelf')
graph.add(node='bowl', parent='drawer')
graph.add(node='pills', parent='drawer')
graph.add(node='banana', parent='counter')
graph.add(node='glass', parent='counter')
graph.add(node='milk', parent='fridge')

graph.draw()

In [ ]:
proactivity_window = 5

prev_edges = graph.get_edges(node_name,node_id)
prev_time = 60

batch = {}

batch['edges'] = prev_edges
batch['nodes'] = standard_nodes
batch['y_edges'] = prev_edges
batch['y_nodes'] = standard_nodes
batch['dynamic_edges_mask'] = torch.tensor(np.ones_like(prev_edges)-np.eye((prev_edges.size()[-1])))
batch['dynamic_edges_mask'][:,:7,:] = 0
batch['context'] = time_encoding_func(prev_time)

edges_seq = []

for _ in range(proactivity_window):
    _, details = model.step(batch)
    edg = (details['output_probs']['location']).to(torch.float32)
    gt_tensor = details['gt']['location'][details['evaluate_node']]
    output_tensor = details['output']['location'][details['evaluate_node']]
    input_tensor = details['input']['location'][details['evaluate_node']]
    prev_time += cfg['DATA_INFO']['dt']
    edges_seq.append(edg)
    batch['edges'] = edg
    batch['context'] = time_encoding_func(prev_time)
    
print_changes(prev_edges, edges_seq, node_name)

edg.argmax(-1)


In [ ]:
graph.get_edges(node_name,node_id).argmax(-1)

In [ ]:
print_changes(graph.get_edges(node_name,node_id), edg, node_name)

In [ ]:
data = RoutinesDataset(data_path=os.path.join(directory,'processed'), 
                        time_encoder=time_encoding_func, 
                        batch_size=cfg['BATCH_SIZE'],
                        only_seen_edges = cfg['ONLY_SEEN_EDGES'],
                        max_routines = (50, None))

visualize_conditional_datapoint(model, data.get_single_example_test_loader(), data.node_classes, use_output_nodes=cfg['LEARN_NODES'])
           

In [ ]:
tl = data.get_single_example_test_loader()
tl[0]

In [ ]:
tl.dataset[-1]


## Final

In [6]:
%load_ext autoreload
%autoreload 2
from DemoRoutine import evaluate_model, generate_data, Graph, print_time, get_initial_graph

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# # ! mv data/demo_routines data/demo_routines_old
# generate_data()

# # ! python3 ./readerFileBased.py --path=data/demo_routines
# # ! python3 ./run.py --path=data/demo_routines --name=ours --tags=demo --logs_dir=logs --write_ckpt

In [7]:
# graph = Graph('kitchen')

# graph.add(node='shelf', parent='kitchen', category='Furniture')
# graph.add(node='drawer', parent='kitchen', category='Furniture')
# graph.add(node='sink', parent='kitchen', category='Furniture')
# graph.add(node='counter', parent='kitchen', category='Furniture')
# graph.add(node='fridge', parent='kitchen', category='Furniture')
# graph.add(node='table', parent='kitchen', category='Furniture')

# graph.add(node='cereal', parent='shelf')
# graph.add(node='bowl', parent='drawer')
# graph.add(node='pills', parent='drawer')
# graph.add(node='waterbottle', parent='counter')
# graph.add(node='glass', parent='counter')
# graph.add(node='milk', parent='fridge')

graph = get_initial_graph()

time = 40

print_time(time)

'06:20'

In [11]:
evaluate_model(graph, time)

graph.move('bowl', 'table')
time += 20


In [12]:
graph.move('bowl', 'table')
graph.move('milk', 'table')
time += 20

In [13]:
print_time(time)

'06:40'

graph.move('cereal', 'table')
graph.move('bowl', 'table')
graph.move('milk', 'table')
time += 20

graph.move('glass', 'table')
time += 20

graph.move('milk', 'fridge')
graph.move('cereal', 'shelf')
time += 20

graph.move('bowl', 'sink')
graph.move('glass', 'sink')
time += 20

graph.move('pills', 'table')
time += 20

time += 20

graph.move('pills', 'drawer')
time += 20